# 대화 상태 (Conversation State)
모델 상호작용 중 대화 상태를 관리하는 방법을 알아보세요.
OpenAI는 대화 상태를 관리하는 몇 가지 방법을 제공하며, 이는 대화에서 여러 메시지 또는 차례에 걸쳐 정보를 보존하는 데 중요합니다.

각 텍스트 생성 요청은 독립적이고 상태가 지정되지 않지만(Assistants API를 사용하는 경우 제외), 텍스트 생성 요청에 추가 메시지를 매개변수로 제공하여 여러 차례에 걸친 대화를 구현할 수 있습니다. 

### 대화 상태 수동 관리 - chat.completions API

사용자 메시지와 어시스턴트 메시지를 번갈아 사용하면 모델에 대한 한 번의 요청으로 대화의 이전 상태를 캡처할 수 있습니다.  
생성된 응답 간에 컨텍스트를 수동으로 공유하려면 모델의 이전 응답 출력을 입력으로 포함하고 해당 입력을 다음 요청에 추가하세요.  
이렇게 이전 응답을 새 요청에 추가하면 대화가 자연스럽게 느껴지고 이전 상호작용의 컨텍스트가 유지됩니다.

### 대화 상태용 OpenAI API
response API를 사용하면 대화 상태를 자동으로 더 쉽게 관리할 수 있으므로 대화가 진행될 때마다 수동으로 입력을 전달할 필요가 없습니다.  
Responses API는 상태를 저장하므로 대화 전반의 컨텍스트 관리가 간단한 매개변수로 가능합니다.

### 문서/텍스트를 기반으로 챗봇이 답변하도록 하기

문서/텍스트 내용을 API 호출시 넘겨주면 우리가 원하는 답변을 유도할 수 있습니다.

------------------
(대화 예시)
```
안녕하세요 고객님. 무엇을 도와드릴까요?
사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 검사 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 해당 필드 내에서 사전 클릭 옵션을 선택하시면 됩니다.
사용자 입력:  인공지능 연구소는 어떤 비밀번호 형식을 요구하나요 
assistant: 인공지능 연구소는 강력한 비밀번호 사용을 권장하며, 비밀번호는 최소 8자 이상이어야 하고 문자와 숫자를 조합해야 합니다.
사용자 입력:  각 섹션의 모든 정보를 제공해야하나요
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 별표(*)로 표시된 모든 항목이 필수 정보로 지정되어 있으므로, 각 섹션의 모든 필수 항목을 반드시 작성해야 합니다.
사용자 입력:  강남에서 남산에 가는 교통편을 알려주세요 
assistant: 강남에서 남산으로 가는 교통편은 여러 가지가 있습니다. 대표적인 방법은 다음과 같습니다:

1. **지하철 이용**:
   - 강남역에서 2호선을 타고 '역삼역' 또는 '선릉역'에서 3호선으로 환승 → 충무로역 또는 명동역에서 하차 → 남산 방향으로 도보 혹은 버스 이용
   - 또는 강남역에서 9호선으로 환승해 '명동역' 또는 '남산촌차고지'로 이동하는 방법도 있습니다.

2. **버스 이용**:
   - 강남 일대에서 남산 순환버스(남산순환버스 또는 시내버스)를 이용할 수 있으며, 일부 버스는 남산 근처 정류장에 정차합니다.

3. **택시 또는 차량 이용**:
   - 택시를 타거나 개인 차량으로 이동하면 약 20~30분 정도 소요됩니다. 남산공원 및 케이블카 하차장 근처에 주차하거나, 케이블카를 이용하려면 남산케이블카 승강장 부근으로 이동하시면 됩니다.

시간과 교통 상황에 따라 최적의 경로를 선택하실 수 있으니, 출발 전에 교통정보 앱이나 지도 서비스를 확인하시는 것을 추천드립니다.
사용자 입력:  quit
```

### 컨텍스트 창 관리
컨텍스트 창을 이해하면 스레드된 대화를 성공적으로 생성하고 모델 상호작용 전반의 상태를 관리하는 데 도움이 됩니다.   
컨텍스트 창은 단일 요청에서 사용할 수 있는 최대 토큰 수입니다. 이 최대 토큰 수에는 입력, 출력 및 추론 토큰이 포함됩니다. 모델의 컨텍스트 창에 대한 자세한 내용은 모델 세부 정보를 참조하세요. 

다음 토큰 수가 컨텍스트 창 총계에 적용됩니다.
- 입력 토큰( Responses APIinput 에 대한 배열 에 포함하는 입력)  
- 출력 토큰(프롬프트에 대한 응답으로 생성된 토큰)  
- 추론 토큰(모델이 응답을 계획하는 데 사용됨)
  
컨텍스트 창 제한을 초과하여 생성된 토큰은 API 응답에서 잘릴 수 있습니다.

<img src="https://i.imgur.com/WcWPlmZ.png" width=500 />

## 실습 문제: "대화 흐름과 상태 유지"

다음의 요구사항에 따라, `OpenAI Responses API`를 사용하여 사용자의 대화 요청에 대해 **대화 흐름을 유지하면서 자연스럽게 응답하는 챗봇**을 만들어 보세요.

### 실습 시나리오

당신은 `디지털 헬스 상담 챗봇`을 만들고자 합니다. 사용자가 건강과 관련된 여러 질문을 연속적으로 했을 때, **이전 응답을 기억하고 연관성 있게 대화**를 이어가야 합니다.


### 주어진 초기 설정

* 사용자 첫 질문:
  `"요즘 소화가 잘 안돼요. 어떻게 해야 할까요?"`

* 이어지는 질문:
  `"그럼 매운 음식은 피해야 하나요?"`


### 구현 조건

1. Responses API를 사용하여 첫 번째 질문에 답변을 생성하세요.
2. `previous_response_id`를 활용하여 두 번째 질문을 **동일한 컨텍스트 흐름**으로 이어서 응답하도록 하세요.
3. 두 번째 응답은 **첫 번째 대답을 기억한 상태**에서 이어지는 조언처럼 들려야 합니다.
4. 사용자의 질문 외에 추가된 답변이 없도록 하세요 (챗봇이 혼자 말하지 않기).
5. 응답의 마지막에 챗봇이 “다른 증상은 없으신가요?” 와 같이 자연스럽게 다음 질문을 유도하도록 만들어 보세요.